In [2]:
import numpy as np
import os
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [3]:
# Load images and labels
def load_images_and_labels(image_dir, classes, img_size):
    images = []
    labels = []
    for label, cls in enumerate(classes):
        class_dir = os.path.join(image_dir, cls)
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            try:
                img = load_img(img_path, target_size=img_size)
                img_array = img_to_array(img)
                images.append(img_array)
                labels.append(label)
            except Exception as e:
                print(f"Error loading image {img_file}: {e}")
    return np.array(images), np.array(labels)

In [4]:
# Load and preprocess training and validation data
train_folder = '/kaggle/input/disasterclassification/train'
val_folder = '/kaggle/input/disasterclassification/validation'

In [5]:
classes = ['earthquake', 'cyclone', 'flood', 'wildfire']
img_size = (224, 224)

In [6]:
print("Loading images...")
train_images, y_train = load_images_and_labels(train_folder, classes, img_size)
val_images, y_val = load_images_and_labels(val_folder, classes, img_size)

# Preprocess images for VGG16
train_images = preprocess_input(train_images)
val_images = preprocess_input(val_images)

# Extract features using VGG16
print("Extracting features...")
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
train_features = vgg_model.predict(train_images)
val_features = vgg_model.predict(val_images)
X_train = train_features.reshape(train_features.shape[0], -1)  # Flatten features
X_val = val_features.reshape(val_features.shape[0],-1)

Loading images...
Extracting features...
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
50/50 ━━━━━━━━━━━━━━━━━━━━ 452s 9s/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 112s 9s/step


In [7]:
print("Training Bagging Classifier...")
bagging_model = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, random_state=42)
bagging_model.fit(X_train, y_train)

y_pred = bagging_model.predict(X_val)
print("Classification Report:")
print(classification_report(y_val, y_pred, target_names=classes))
print(f"Accuracy: {accuracy_score(y_val, y_pred):.2f}")

Training Bagging Classifier...


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Classification Report:
              precision    recall  f1-score   support

  earthquake       0.90      0.94      0.92       100
     cyclone       0.93      0.93      0.93       100
       flood       0.96      0.92      0.94       100
    wildfire       0.90      0.90      0.90       100

    accuracy                           0.92       400
   macro avg       0.92      0.92      0.92       400
weighted avg       0.92      0.92      0.92       400

Accuracy: 0.92


In [8]:
import numpy as np
import os
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Train AdaBoost Classifier
print("Training AdaBoost Classifier...")
base_learner = DecisionTreeClassifier(max_depth=1)  # Weak learner
boosting_model = AdaBoostClassifier(base_estimator=base_learner, n_estimators=150, learning_rate=0.1, random_state=42)
boosting_model.fit(X_train, y_train)

# Evaluate the model
y_pred = boosting_model.predict(X_val)
print("Classification Report:")
print(classification_report(y_val, y_pred, target_names=classes))
print(f"Accuracy: {accuracy_score(y_val, y_pred):.2f}")

Training AdaBoost Classifier...


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Classification Report:
              precision    recall  f1-score   support

  earthquake       0.99      0.76      0.86       100
     cyclone       0.99      0.91      0.95       100
       flood       0.80      0.96      0.87       100
    wildfire       0.86      0.96      0.91       100

    accuracy                           0.90       400
   macro avg       0.91      0.90      0.90       400
weighted avg       0.91      0.90      0.90       400

Accuracy: 0.90


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from itertools import product

def grid_search(X_train, y_train, X_val, y_val, param_grid):
    """
    Perform grid search to find the best hyperparameters for AdaBoost.

    Args:
        X_train: Features for training.
        y_train: Labels for training.
        X_val: Features for validation.
        y_val: Labels for validation.
        param_grid: Dictionary of hyperparameters and their values to search.

    Returns:
        best_params: Dictionary of the best parameters.
        best_model: The best AdaBoost model trained with the best parameters.
        best_score: The highest validation accuracy.
    """
    best_score = 0
    best_params = None
    best_model = None

    # Generate all combinations of hyperparameters
    keys, values = zip(*param_grid.items())
    for param_combination in product(*values):
        params = dict(zip(keys, param_combination))

        print(params)
        # Initialize AdaBoost with the current set of parameters
        base_learner = DecisionTreeClassifier(max_depth=params['max_depth'])
        model = AdaBoostClassifier(
            estimator=base_learner,
            n_estimators=params['n_estimators'],
            learning_rate=params['learning_rate'],
            random_state=42
        )
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Validate the model
        y_val_pred = model.predict(X_val)
        score = accuracy_score(y_val, y_val_pred)
        print(score)
        
        # Update the best model if the current one is better
        if score > best_score:
            best_score = score
            best_params = params
            best_model = model
    
    return best_params, best_model, best_score

In [10]:
param_grid = {
    'n_estimators': [80, 150, 200],
    'learning_rate': [0.01, 0.1, 1.0],
    'max_depth': [1, 2, 3]
}

# Perform grid search
print("Performing Grid Search...")
best_params, best_model, best_score = grid_search(X_train, y_train, X_val, y_val, param_grid)

print(f"Best Parameters: {best_params}")
print(f"Validation Accuracy: {best_score:.4f}")


Performing Grid Search...
{'n_estimators': 80, 'learning_rate': 0.01, 'max_depth': 1}
0.7625
{'n_estimators': 80, 'learning_rate': 0.01, 'max_depth': 2}
0.8325
{'n_estimators': 80, 'learning_rate': 0.01, 'max_depth': 3}
0.855
{'n_estimators': 80, 'learning_rate': 0.1, 'max_depth': 1}
0.9025
{'n_estimators': 80, 'learning_rate': 0.1, 'max_depth': 2}
0.935
{'n_estimators': 80, 'learning_rate': 0.1, 'max_depth': 3}
0.905
{'n_estimators': 80, 'learning_rate': 1.0, 'max_depth': 1}
0.855
{'n_estimators': 80, 'learning_rate': 1.0, 'max_depth': 2}
0.875
{'n_estimators': 80, 'learning_rate': 1.0, 'max_depth': 3}
0.9475
{'n_estimators': 150, 'learning_rate': 0.01, 'max_depth': 1}
0.7875
{'n_estimators': 150, 'learning_rate': 0.01, 'max_depth': 2}
0.8675
{'n_estimators': 150, 'learning_rate': 0.01, 'max_depth': 3}
0.8875
{'n_estimators': 150, 'learning_rate': 0.1, 'max_depth': 1}
0.9225
{'n_estimators': 150, 'learning_rate': 0.1, 'max_depth': 2}
0.95
{'n_estimators': 150, 'learning_rate': 0.1, 'm